# **Joining and merging** 

It  allow combining DataFrames similarly to SQL joins, primarily using the `merge()` function and `.join()` method. The four core pillars are:

1. Functions: `merge()` (flexible, SQL-like joins on columns or index) and `.join()` (joins on index by default).

2. Join types: inner, left, right, and outer define which rows are retained during the join.

3. Keys: columns or indexes on which the join is based, supporting single or multiple keys.

4. Common pitfalls: mismatched keys, duplicated column names, and misunderstanding index vs column joining.

In realworld scenarios data are often spread accross multiple files, Datbases or even APIS. combining this data allows us to complete picture and and perform meaningful analysis. 

# Parent and Child Tables

Parent and child tables represent a **"one-to-many"** data relationship.  
- The **parent table** holds unique records with a **primary key**.  
- The **child table** holds related records referencing the parent's key as a **foreign key**.


# Types of Joins

When joining tables:

- **Inner join** returns only rows with matching keys in both tables.
- **Left join** returns all rows from the left table and matched rows from the right table. Rows from the left table with no match in the right will have `null` or `NaN` in the right table columns.
- **Right join** returns all rows from the right table and matched rows from the left table. Rows from the right table with no match in the left will have `null` or `NaN` in the left table columns.
- **Outer join** returns **all rows** from both tables, matching where possible, and filling with `null` or `NaN` where no match exists.

> **Note:**  
> Which table is left or right depends on the join operation, and either the parent or child table can be on either side depending on the context.

# **Relating to SQL**

| Join Type   | SQL Parameter | Pandas `merge()` `how` Parameter |
|-------------|---------------|----------------------------------|
| Inner Join  | `INNER JOIN`  | `how='inner'`                    |
| Left Join   | `LEFT JOIN`   | `how='left'`                     |
| Right Join  | `RIGHT JOIN`  | `how='right'`                    |
| Outer Join  | `FULL OUTER JOIN` | `how='outer'`                   |


### Table A (Penguins Basic)

| species   | island    | body_mass_g |
|-----------|-----------|-------------|
| Adelie    | Torgersen | 3750        |
| Chinstrap | Dream     | 3800        |
| Gentoo    | Biscoe    | 5000        |

### Table B (Penguins Extra Info)

| species   | flipper_length_mm |
|-----------|-------------------|
| Adelie    | 181               |
| Gentoo    | 220               |
| Macaroni  | 195               |

---

### Inner Join on `species` (only matching species)

| species   | island    | body_mass_g | flipper_length_mm |
|-----------|-----------|-------------|-------------------|
| Adelie    | Torgersen | 3750        | 181               |
| Gentoo    | Biscoe    | 5000        | 220               |

---

### Left Join (all from Table A, matched from Table B)

| species   | island    | body_mass_g | flipper_length_mm |
|-----------|-----------|-------------|-------------------|
| Adelie    | Torgersen | 3750        | 181               |
| Chinstrap | Dream     | 3800        | NULL              |
| Gentoo    | Biscoe    | 5000        | 220               |

---

### Right Join (all from Table B, matched from Table A)

| species   | island    | body_mass_g | flipper_length_mm |
|-----------|-----------|-------------|-------------------|
| Adelie    | Torgersen | 3750        | 181               |
| Gentoo    | Biscoe    | 5000        | 220               |
| Macaroni  | NULL      | NULL        | 195               |

---

### Outer Join (all from both, matched where possible)

| species   | island    | body_mass_g | flipper_length_mm |
|-----------|-----------|-------------|-------------------|
| Adelie    | Torgersen | 3750        | 181               |
| Chinstrap | Dream     | 3800        | NULL              |
| Gentoo    | Biscoe    | 5000        | 220               |
| Macaroni  | NULL      | NULL        | 195               |


# **Best Practise For Joins**
1. Specify the parent Table first. 
2. Validate keys. 
3. Inspect missing Data. 

In [ ]:
import pandas as pd
penguins=pd.read_csv("../Datasets/penguins.csv")   
penguins.head()

,rowid,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
0,1,Adelie,Torgersen,39.1,18.7,181.0,3750.0,male,2007
1,2,Adelie,Torgersen,39.5,17.4,186.0,3800.0,female,2007
2,3,Adelie,Torgersen,40.3,18.0,195.0,3250.0,female,2007
3,4,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,2007
4,5,Adelie,Torgersen,36.7,19.3,193.0,3450.0,female,2007


In [ ]:
# Prep Splits for joining (Simulate tables) 

left=penguins[["rowid","species","island","sex","year"]].copy()
right=penguins[["rowid","bill_length_mm","bill_depth_mm","flipper_length_mm","body_mass_g"]].copy()

# Inject mismatches
right=right.drop([0,10,20])     #Arbitrary drops for demo—change to random for variation."
right=right.dropna(subset=["bill_length_mm"])

# inner join left and right (binary)
inner_merged = left.merge(right, on="rowid", how='inner')
print(inner_merged.head())
print(inner_merged.isna().sum())  # Check for NaNs (e.g., sex might have some)

inner_merged.groupby("species")["body_mass_g"].mean()

   rowid species     island     sex  year  bill_length_mm  bill_depth_mm  \
0      2  Adelie  Torgersen  female  2007            39.5           17.4   
1      3  Adelie  Torgersen  female  2007            40.3           18.0   
2      5  Adelie  Torgersen  female  2007            36.7           19.3   
3      6  Adelie  Torgersen    male  2007            39.3           20.6   
4      7  Adelie  Torgersen  female  2007            38.9           17.8   

   flipper_length_mm  body_mass_g  
0              186.0       3800.0  
1              195.0       3250.0  
2              193.0       3450.0  
3              190.0       3650.0  
4              181.0       3625.0  
rowid                0
species              0
island               0
sex                  8
year                 0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
dtype: int64


species
Adelie       3705.067568
Chinstrap    3733.088235
Gentoo       5076.016260
Name: body_mass_g, dtype: float64

In [51]:
# Multi-Column Inner (No row_id Needed): For species-year combos.
left_multi = penguins[['species', 'island', 'sex', 'year']]
right_multi = penguins[['species', 'year', 'bill_length_mm', 'body_mass_g']].drop([0, 10])  # Mismatches

inner_multi = pd.merge(left_multi, right_multi, on=['species', 'year'], how='inner')
print(inner_multi.head())  # Fewer rows if year-species don't overlap perfectly

  species     island   sex  year  bill_length_mm  body_mass_g
0  Adelie  Torgersen  male  2007            39.5       3800.0
1  Adelie  Torgersen  male  2007            40.3       3250.0
2  Adelie  Torgersen  male  2007             NaN          NaN
3  Adelie  Torgersen  male  2007            36.7       3450.0
4  Adelie  Torgersen  male  2007            39.3       3650.0


# Inner Join in Pandas

1. **Understanding Inner Joins**  
   Inner joins combine datasets on a common key.  
   Only rows with matching keys in both datasets are included.  
   Example: Finding sales reps who have made sales, including only matching records.

2. **Performing an Inner Join in Pandas**  
   Use `pd.merge()` to combine datasets.  
   Specify join keys using `left_on` and `right_on` parameters (or `on` if key name is the same).

3. **Practical Use Case: Aggregating mass by Species**  
   After merging, use operations like `groupby()` for aggregation.  
   Example: Compute total sales by region using `groupby()`.

4. **Best Practices for Inner Joins**  
   - ✅ Ensure data consistency: Confirm join keys are properly defined.  
   - ✅ Inspect results: Use `.shape` to check row counts after join.  
   - ✅ Handle missing keys: Remember unmatched rows are excluded; consider left/right join if you want to keep them.

In [58]:
# Left join: Keep all left, match right on rowid
left_join=pd.merge(left,right, on="rowid", how="left")
print(left_join.head(5))
print(left_join.isna().sum())

   rowid species     island     sex  year  bill_length_mm  bill_depth_mm  \
0      1  Adelie  Torgersen    male  2007             NaN            NaN   
1      2  Adelie  Torgersen  female  2007            39.5           17.4   
2      3  Adelie  Torgersen  female  2007            40.3           18.0   
3      4  Adelie  Torgersen     NaN  2007             NaN            NaN   
4      5  Adelie  Torgersen  female  2007            36.7           19.3   

   flipper_length_mm  body_mass_g  
0                NaN          NaN  
1              186.0       3800.0  
2              195.0       3250.0  
3                NaN          NaN  
4              193.0       3450.0  
rowid                 0
species               0
island                0
sex                  11
year                  0
bill_length_mm        5
bill_depth_mm         5
flipper_length_mm     5
body_mass_g           5
dtype: int64


In [59]:
# Multi-Column Left Join (Composite Keys, No rowid)

left_multi = penguins[['species', 'island', 'sex', 'year']].copy()
right_multi = penguins[['species', 'year', 'bill_length_mm', 'body_mass_g']].drop([0, 10]).copy()

left_multi_join = pd.merge(left_multi, right_multi, on=['species', 'year'], how='left')
print(left_multi_join.shape)  # (344, 6) — All left, duplicates if multiple matches per key
print(left_multi_join.head())

(14388, 6)
  species     island   sex  year  bill_length_mm  body_mass_g
0  Adelie  Torgersen  male  2007            39.5       3800.0
1  Adelie  Torgersen  male  2007            40.3       3250.0
2  Adelie  Torgersen  male  2007             NaN          NaN
3  Adelie  Torgersen  male  2007            36.7       3450.0
4  Adelie  Torgersen  male  2007            39.3       3650.0


In [64]:
# Index-Based Left Join

left_index = left.set_index('rowid')
right_index = right.set_index('rowid')

left_index_join = pd.merge(left_index, right_index, left_index=True, right_index=True, how='left')
left_index_join.reset_index(inplace=True)
print(left_index_join.shape) 
left_index_join

(344, 9)


,rowid,species,island,sex,year,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,1,Adelie,Torgersen,male,2007,NaN,NaN,NaN,NaN
1,2,Adelie,Torgersen,female,2007,39.5,17.4,186.0,3800.0
2,3,Adelie,Torgersen,female,2007,40.3,18.0,195.0,3250.0
3,4,Adelie,Torgersen,NaN,2007,NaN,NaN,NaN,NaN
4,5,Adelie,Torgersen,female,2007,36.7,19.3,193.0,3450.0
...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,male,2009,55.8,19.8,207.0,4000.0
340,341,Chinstrap,Dream,female,2009,43.5,18.1,202.0,3400.0
341,342,Chinstrap,Dream,male,2009,49.6,18.2,193.0,3775.0
342,343,Chinstrap,Dream,male,2009,50.8,19.0,210.0,4100.0


In [63]:
# right_join
right_join=pd.merge(left,right,on="rowid",how="right")
right_join

,rowid,species,island,sex,year,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
0,2,Adelie,Torgersen,female,2007,39.5,17.4,186.0,3800.0
1,3,Adelie,Torgersen,female,2007,40.3,18.0,195.0,3250.0
2,5,Adelie,Torgersen,female,2007,36.7,19.3,193.0,3450.0
3,6,Adelie,Torgersen,male,2007,39.3,20.6,190.0,3650.0
4,7,Adelie,Torgersen,female,2007,38.9,17.8,181.0,3625.0
...,...,...,...,...,...,...,...,...,...
334,340,Chinstrap,Dream,male,2009,55.8,19.8,207.0,4000.0
335,341,Chinstrap,Dream,female,2009,43.5,18.1,202.0,3400.0
336,342,Chinstrap,Dream,male,2009,49.6,18.2,193.0,3775.0
337,343,Chinstrap,Dream,male,2009,50.8,19.0,210.0,4100.0


In [67]:
unfound=left_join[left_join["sex"].isnull()]
unfound

,rowid,species,island,sex,year,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
3,4,Adelie,Torgersen,NaN,2007,NaN,NaN,NaN,NaN
8,9,Adelie,Torgersen,NaN,2007,34.1,18.1,193.0,3475.0
9,10,Adelie,Torgersen,NaN,2007,42.0,20.2,190.0,4250.0
10,11,Adelie,Torgersen,NaN,2007,NaN,NaN,NaN,NaN
11,12,Adelie,Torgersen,NaN,2007,37.8,17.3,180.0,3700.0
47,48,Adelie,Dream,NaN,2007,37.5,18.9,179.0,2975.0
178,179,Gentoo,Biscoe,NaN,2007,44.5,14.3,216.0,4100.0
218,219,Gentoo,Biscoe,NaN,2008,46.2,14.4,214.0,4650.0
256,257,Gentoo,Biscoe,NaN,2009,47.3,13.8,216.0,4725.0
268,269,Gentoo,Biscoe,NaN,2009,44.5,15.7,217.0,4875.0


In [ ]:
# query and get  results
left_join.query("sex.isnull()")

,rowid,species,island,sex,year,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
3,4,Adelie,Torgersen,NaN,2007,NaN,NaN,NaN,NaN
8,9,Adelie,Torgersen,NaN,2007,34.1,18.1,193.0,3475.0
9,10,Adelie,Torgersen,NaN,2007,42.0,20.2,190.0,4250.0
10,11,Adelie,Torgersen,NaN,2007,NaN,NaN,NaN,NaN
11,12,Adelie,Torgersen,NaN,2007,37.8,17.3,180.0,3700.0
47,48,Adelie,Dream,NaN,2007,37.5,18.9,179.0,2975.0
178,179,Gentoo,Biscoe,NaN,2007,44.5,14.3,216.0,4100.0
218,219,Gentoo,Biscoe,NaN,2008,46.2,14.4,214.0,4650.0
256,257,Gentoo,Biscoe,NaN,2009,47.3,13.8,216.0,4725.0
268,269,Gentoo,Biscoe,NaN,2009,44.5,15.7,217.0,4875.0


# Understanding Left and Right Joins

## Left Join:
- Retains all rows from the **left table** (parent dataset).
- Matches records from the **right table** (child dataset).
- Unmatched rows in the right table are filled with **NaN**.

## Right Join:
- Retains all rows from the **right table** (child dataset).
- Matches records from the **left table** (parent dataset).
- Unmatched rows in the left table are filled with **NaN**.

---

# ** Outer Joins: Full outer Join**

It keeps everything from both sides like a union with Nans for gaps. This is perfect for spotting data discrepancies but it can bloat with NaNs. 


In [77]:
left = penguins[["rowid", "species", "island", "sex", "year"]].copy()
right = penguins[["rowid", "bill_length_mm", "bill_depth_mm", "flipper_length_mm", "body_mass_g"]].copy()
right = right.drop([0, 10, 20]).dropna(subset=["bill_length_mm"])

In [78]:
outer_merged=pd.merge(left,right,on="rowid",how="outer", indicator=True) # Indicator flags sources
print(outer_merged.head(5))
print(outer_merged['_merge'].value_counts())  # both: matches, left_only: right gaps, right_only: left gaps
print(outer_merged.isna().sum())  # NaNs in gaps 


   rowid species     island     sex  year  bill_length_mm  bill_depth_mm  \
0      1  Adelie  Torgersen    male  2007             NaN            NaN   
1      2  Adelie  Torgersen  female  2007            39.5           17.4   
2      3  Adelie  Torgersen  female  2007            40.3           18.0   
3      4  Adelie  Torgersen     NaN  2007             NaN            NaN   
4      5  Adelie  Torgersen  female  2007            36.7           19.3   

   flipper_length_mm  body_mass_g     _merge  
0                NaN          NaN  left_only  
1              186.0       3800.0       both  
2              195.0       3250.0       both  
3                NaN          NaN  left_only  
4              193.0       3450.0       both  
_merge
both          339
left_only       5
right_only      0
Name: count, dtype: int64
rowid                 0
species               0
island                0
sex                  11
year                  0
bill_length_mm        5
bill_depth_mm         5
flippe

In [79]:
#clean-up data
outer_merged.fillna({'sex': 'Unknown', 'bill_length_mm': 0})

,rowid,species,island,sex,year,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,_merge
0,1,Adelie,Torgersen,male,2007,0.0,NaN,NaN,NaN,left_only
1,2,Adelie,Torgersen,female,2007,39.5,17.4,186.0,3800.0,both
2,3,Adelie,Torgersen,female,2007,40.3,18.0,195.0,3250.0,both
3,4,Adelie,Torgersen,Unknown,2007,0.0,NaN,NaN,NaN,left_only
4,5,Adelie,Torgersen,female,2007,36.7,19.3,193.0,3450.0,both
...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,male,2009,55.8,19.8,207.0,4000.0,both
340,341,Chinstrap,Dream,female,2009,43.5,18.1,202.0,3400.0,both
341,342,Chinstrap,Dream,male,2009,49.6,18.2,193.0,3775.0,both
342,343,Chinstrap,Dream,male,2009,50.8,19.0,210.0,4100.0,both


In [ ]:
# Drop Nan=Heavy: 

outer_merged.dropna(subset=['bill_length_mm', 'species'])

,rowid,species,island,sex,year,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,_merge
1,2,Adelie,Torgersen,female,2007,39.5,17.4,186.0,3800.0,both
2,3,Adelie,Torgersen,female,2007,40.3,18.0,195.0,3250.0,both
4,5,Adelie,Torgersen,female,2007,36.7,19.3,193.0,3450.0,both
5,6,Adelie,Torgersen,male,2007,39.3,20.6,190.0,3650.0,both
6,7,Adelie,Torgersen,female,2007,38.9,17.8,181.0,3625.0,both
...,...,...,...,...,...,...,...,...,...,...
339,340,Chinstrap,Dream,male,2009,55.8,19.8,207.0,4000.0,both
340,341,Chinstrap,Dream,female,2009,43.5,18.1,202.0,3400.0,both
341,342,Chinstrap,Dream,male,2009,49.6,18.2,193.0,3775.0,both
342,343,Chinstrap,Dream,male,2009,50.8,19.0,210.0,4100.0,both


| Join Type | Rows Kept                              | Use Case             | NaNs Present?         | Approx. Penguin Rows Kept |
|-----------|--------------------------------------|----------------------|-----------------------|----------------------------|
| Inner     | Matches only                         | Clean intersections  | No                    | 339                        |
| Left      | All rows from left + matches from right | Preserve main data   | Yes (right gaps)      | 344                        |
| Right     | All rows from right + matches from left | Preserve secondary   | Yes (left gaps)       | 339                        |
| Outer     | All rows from both tables            | Full audit           | Yes (both sides)      | 344                        |

---

- Inner Join: Returns only rows with keys present in both DataFrames, no missing values, fewer rows.

- Left Join: Returns all rows from the left DataFrame (e.g., main penguins info), with missing matches filled as NaN on the right.

- Right Join: Returns all rows from the right DataFrame (e.g., supplementary measurements), with missing matches from left as NaN.

- Outer Join: Returns all rows from both DataFrames, including all unmatched rows from both sides, so most complete but most NaNs.